### Aggregating synthpop synthetic populations to tract level
1. read in individual csv files containing synthetic populations
2. combine into one large dataframe
3. group by tract to aggregate counts
4. compute new implied variable
5. export as csv to join to tract boundaries in gis

In [8]:
import pandas as pd
import csv
import numpy as np
# library for path names
import glob 
from functools import reduce
pd.options.display.max_columns = 40

In [3]:
# function to compiling dataframe from folder of individual csv files
def compile_df(pathfile_name):
    li = []
    all_files = glob.glob(pathfile_name)
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, converters={'tract': lambda x: str(x)})
        li.append(df)
    return(pd.concat(li, axis=0, ignore_index=True))

### Aggregating household-level synthetic population

In [51]:
## creating household level variables/tables
households_meck_comp = compile_df("data_outputs/20190323_run/*households.csv")
households_meck_comp.to_csv("data_outputs/20190323_compiled/households_meck.csv")

# converting columns to correct data type
households_meck = pd.read_csv("data_outputs/20190323_compiled/households_meck.csv", 
                       header=0, dtype={'puma00':str, 'puma10':str, 'NP':int,
                                        'TYPE':int,'race_of_head':str,
                                        'hispanic_head':str, 'age_of_head':int,
                                        'hh_age_of_head':str, 'hh_cars':str,
       'hh_children':str, 'hh_income':str, 'hh_race_of_head':str, 'hh_size':str, 'hh_workers':str,
       'seniors':str, 'sf_detached':str, 'tenure_mover':str,'state':str, 'county':str,
       'tract':str, 'block group':str  
                       })
## dropping variables we are not using
households_var = households_meck.drop(columns=['Unnamed: 0.1', 'serialno', 'RT', 'puma00', 'puma10',
       'NP', 'TYPE', 'BLD', 'TEN', 'VEH', 'HINCP', 'MV', 'R18', 'R65',
       'race_of_head', 'hispanic_head', 'age_of_head', 'workers',
       'hh_age_of_head', 'hh_children', 'hh_race_of_head',
       'hh_size', 'hh_workers', 'seniors', 'sf_detached', 'tenure_mover',
       'cat_id', 'state', 'county', 'block group'])

## aggregating households by cars and tract
households_meck_tract_cars = households_var.groupby(["tract","hh_cars"]).count()
households_meck_tract_income = households_var.groupby(["tract","hh_income"]).count()
households_meck_tract_tot = households_var.groupby(["tract"]).count()

# resetting index and unstacking to make a wide data frame
house_cars = households_meck_tract_cars.reset_index().pivot(index='tract', columns='hh_cars', values ='Unnamed: 0').fillna(0)
house_income = households_meck_tract_income.reset_index().pivot(index='tract', columns='hh_income', values ='Unnamed: 0').fillna(0) 
house_tot = households_meck_tract_tot.drop(columns={'hh_cars','hh_income'}).rename(columns={'Unnamed: 0':'total'})
# households_synth = house_cars.merge(house_income,how='outer', on="tract")
households = [house_tot,house_income,house_cars]
households_synth = reduce(lambda left,right: pd.merge(left,right,on="tract"),households)


##adding prefix to columns
households_synth.columns = ['s_' + str(col) for col in households_synth.columns]



In [47]:
households_meck_tract_tot = households_var.groupby(["tract"]).count()
households_meck_tract_tot.head()

,Unnamed: 0,hh_cars,hh_income
tract,,,
000100,2820,2820,2820
000300,481,481,481
000400,1762,1762,1762
000500,2492,2492,2492
000600,1567,1567,1567


### Aggregating person-level synthetic population

In [32]:
# person level data
persons_meck_comp = compile_df("data_outputs/20190323_run/*persons.csv")
persons_meck_comp.to_csv("data_outputs/20190323_compiled/persons_meck.csv")
persons_meck = pd.read_csv("data_outputs/20190323_compiled/persons_meck.csv",
                                 header=0, dtype = {'AGEP':int,
       'RELP':int, 'SEX':str,'HISP':str, 'RAC1P':str,
       'hispanic':str, 'person_age':str, 'person_sex':str, 'race':str,
       'state':str, 'county':str, 'tract':str, 'block group':str})

# # aggregating for total pop person level data
persons_var = persons_meck.drop(columns=['Unnamed: 0.1', 'serialno', 'SPORDER', 'puma00', 'puma10',
       'AGEP', 'JWTR', 'RELP', 'SCH', 'SCHL', 'SEX', 'WKHP', 'ESR', 'HISP',
       'PERNP', 'RAC1P',
       'cat_id', 'hh_id', 'state', 'county', 'block group'])
persons_tract_tot = persons_var.groupby(["tract"]).count()
persons_tract_age = persons_var.groupby(["tract","person_age"]).count()
persons_tract_sex = persons_var.groupby(["tract","person_sex"]).count()
persons_tract_race = persons_var.groupby(["tract","race"]).count()
persons_tract_hisp = persons_var.groupby(["tract","hispanic"]).count()

## resetting index and unstacking to make a wide data frame
persons_tract_tot = persons_tract_tot.drop(columns = ['hispanic','person_age','person_sex',
                                  'race']).rename(columns={'Unnamed: 0':'total'})
persons_tract_age = persons_tract_age.reset_index().pivot(index='tract',columns="person_age", values="Unnamed: 0").dropna(0)
persons_tract_sex = persons_tract_sex.reset_index().pivot(index='tract',columns="person_sex", values="Unnamed: 0").dropna(0)
persons_tract_race = persons_tract_race.reset_index().pivot(index='tract',columns="race", values="Unnamed: 0").dropna(0)
persons_tract_hisp = persons_tract_hisp.reset_index().pivot(index='tract',columns="hispanic",values="Unnamed: 0").dropna(0)
persons = [persons_tract_tot,persons_tract_age, persons_tract_sex, persons_tract_race,persons_tract_hisp ]
persons_synth = reduce(lambda left,right: pd.merge(left,right,on="tract"),persons)

## adding prefix to columns
persons_synth.columns = ['s_' + str(col) for col in persons_synth.columns]
persons_synth= persons_synth.rename(columns = {"s_no":"s_nonhispanic","s_yes":"s_hispanic"})

### Combining with ACS estimates and Margins of error

In [57]:
# loading in acs estimates
ACS_people = pd.read_csv("data_outputs/20190330_census_aggregates/37119_people_meck.csv", converters={'tract': lambda x: str(x)})
ACS_households = pd.read_csv("data_outputs/20190330_census_aggregates/37119_households_meck.csv", converters={'tract': lambda x: str(x)})

In [58]:
# merging acs estimates with synthetics
people = ACS_people.merge(persons_synth,how='outer',on='tract')
households = ACS_households.merge(households_synth, how='outer',
                                  on='tract')

In [59]:
# creating a geo id to assist with gis table join 
people['geoid'] = '37119' + people['tract'].astype(str)
households['geoid'] = '37119' + households['tract'].astype(str)

In [75]:
people.columns


Index(['tract', '19 and under', '19 and under_me', '20 to 35', '20 to 35_me',
       '35 to 60', '35 to 60_me', 'above 60', 'above 60_me', 'nonhispanic',
       'nonhispanic_me', 'hispanic', 'hispanic_me', 'total', 'total_me',
       'asian', 'asian_me', 'black', 'black_me', 'other', 'other_me', 'white',
       'white_me', 'female', 'female_me', 'male', 'male_me', 's_total',
       's_19 and under', 's_20 to 35', 's_35 to 60', 's_above 60', 's_female',
       's_male', 's_asian', 's_black', 's_other', 's_white', 's_nonhispanic',
       's_hispanic', 'geoid', 'd_19und', 'd_202035', 'd_35to60', 'd_60up',
       'd_hispanic', 'd_total', 'd_white', 'd_asian', 'd_black', 'd_other',
       'd_female', 'd_male'],
      dtype='object')

In [61]:
# computing people difference columns
people['d_19und']= people['s_19 and under']-people['19 and under']
people['d_202035']= people['s_20 to 35'] - people['20 to 35'] 
people['d_35to60']= people['s_35 to 60'] - people['35 to 60']
people['d_60up']= people['s_above 60']- people['above 60']
people['d_hispanic']= people['s_hispanic']-people['hispanic']
people['d_total']= people['s_total']- people['total']
people['d_white']= people['s_white']-people['white']
people['d_asian']= people['s_asian']-people['asian']
people['d_black']= people['s_black']-people['black']
people['d_other']= people['s_other']-people['other']
people['d_female']= people['s_female']-people['female']
people['d_male']= people['s_male']-people['male']

In [64]:
# computing household difference columns
households['d_none_car']= households['s_none']-households['none']
households['d_one_car']=households['s_one']-households['one']
households['d_twomore_car']=households['s_two or more']-households['two or more']
households['d_totalhh']= households['s_total']-households['total']
households['d_lt30']=households['s_lt30']-households['lt30']
households['d_gt30-lt60']=households['s_gt30-lt60']-households['gt30-lt60']
households['d_gt60-lt100']=households['s_gt60-lt100']-households['gt60-lt100']
households['d_gt100-lt150']=households['s_gt100-lt150']-households['gt100-lt150']


In [69]:
# exporting combined tables to csv for GIS work
people.to_csv("data_outputs/20190330_ACS_SyntheticsForGIS/37119_people_var_tract.csv")
households.to_csv("data_outputs/20190330_ACS_SyntheticsForGIS/37119_households_var_tract.csv")

In [77]:
households.columns

Index(['tract', 'none', 'none_me', 'one', 'one_me', 'two or more',
       'two or more_me', 'total', 'total_me', 'gt100-lt150', 'gt100-lt150_me',
       'gt150', 'gt150_me', 'gt30-lt60', 'gt30-lt6_me', 'gt60-lt100',
       'gt60-lt100_me', 'lt30', 'lt30_me', 's_total', 's_gt100-lt150',
       's_gt150', 's_gt30-lt60', 's_gt60-lt100', 's_lt30', 's_none', 's_one',
       's_two or more', 'geoid', 'd_none_car', 'd_one_car', 'd_twomore_car',
       'd_totalhh', 'd_lt30', 'd_gt30-lt60', 'd_gt60-lt100', 'd_gt100-lt150'],
      dtype='object')